In [1]:
import os
import pandas as pd
import re
import meshio
import tqdm

In [2]:
df_pos = pd.read_csv('SciVisContest23/viz-no-network/positions/rank_0_positions.txt', sep=' ', names=['local_id', 'x', 'y', 'z', 'area', 'type'], skiprows=8)

In [3]:
df_pos['area'] = df_pos['area'].apply(lambda x: re.search(r'.*_(\d+)', x).group(1))

In [4]:
points = df_pos[['x', 'y', 'z']].values
point_data = {
    'id': df_pos['local_id'].values,
    'area': df_pos['area'].values.astype(int),
}
cells = {}
mesh = meshio.Mesh(points=points, cells=cells, point_data=point_data)
mesh.write('SciVisContest23/viz-no-network/timeseries/rank_0_positions.vtk')

In [5]:
files = os.listdir('SciVisContest23/viz-no-network/network')
files.sort()
i = 0
for file in tqdm.tqdm(files): 
    df_network = pd.read_csv('SciVisContest23/viz-no-network/network/' + file, sep='\s+', 
        names=['target_rank', 'target_id', 'source_rank', 'source_id', 'weight'], skiprows=5)
    connections = list(map(lambda x: x.tolist(), df_network[['source_id', 'target_id']].to_numpy()))
    connections = [[j - 1 for j in el] for el in connections]
    cells = {} if len(connections) == 0 else [
        ('line', connections)
    ]
    mesh = meshio.Mesh(points=points, cells=cells, point_data=point_data)
    mesh.write('SciVisContest23/viz-no-network/timeseries/links/step_' + str(i) + '.vtk')
    i += 1


100%|██████████| 202/202 [06:05<00:00,  1.81s/it]
